# SQL Tables

> Class declarations of the database tables.

In [ ]:
#| default_exp databases

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti
from abc import ABC, abstractmethod
from typing import List

In [ ]:
#| export
class FaceDataBase(ABC):
    def __init__(self, 
    input_dir:str,
    source:str
    ):
        self.input_dir= input_dir
        self.source = source

In [ ]:
b = FaceDataBase('i', 's')
b.__dict__

{'input_dir': 'i', 'source': 's'}

In [ ]:
#| export

class LFW(FaceDataBase):
    def __init__(self, input_dir):
        super().__init__(input_dir, source = 'LFW')
    

In [ ]:
#| export

DATABASES = {'lfw': LFW 
                    # 'xqlfw': XQLFW, 
                    #  'scface': SCFace, 
                    #  'forenface': ForenFace, 
                    #  'utkface': UTKFace,
                    #  'enfsi': Enfsi, 
                    #  'enfsi2015': Enfsi2015
                     }



In [ ]:
#| export
def get_image_db(input_dir:str,
                database_names:List[str]) -> List[FaceDataBase]:
        """
        From str to FaceDataBase class.

        :param scorer_names: List[str]
        :return: List[FaceDataBase]
        """
        # todo: update description.
        # todo: Raise error if str input is not contained in the DB dictionary.
        # DATABASES = {'lfw': LFW, 
                    # 'xqlfw': XQLFW, 
                    #  'scface': SCFace, 
                    #  'forenface': ForenFace, 
                    #  'utkface': UTKFace,
                    #  'enfsi': Enfsi, 
                    #  'enfsi2015': Enfsi2015}
        return [DATABASES[db](input_dir = input_dir) for db in database_names]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()